# Tworzenie agentów AI z trwałą pamięcią za pomocą Cognee

Ten notatnik pokazuje, jak budować inteligentne agenty AI z zaawansowanymi możliwościami pamięci, korzystając z [**cognee**](https://www.cognee.ai/) - otwartoźródłowej pamięci AI, która łączy grafy wiedzy, wyszukiwanie semantyczne i zarządzanie sesjami, aby tworzyć systemy AI świadome kontekstu.

## 🎯 Cele nauki

Po ukończeniu tego samouczka będziesz rozumieć, jak:
- **Budować grafy wiedzy wspierane przez osadzenia**: Przekształcać nieustrukturyzowany tekst w ustrukturyzowaną, zapytaniową wiedzę
- **Wdrażać pamięć sesji**: Tworzyć wieloetapowe rozmowy z automatycznym zachowaniem kontekstu
- **Przechowywać rozmowy**: Opcjonalnie zapisywać ważne interakcje w pamięci długoterminowej do przyszłego wykorzystania
- **Zapytania w języku naturalnym**: Uzyskiwać dostęp do historycznego kontekstu i wykorzystywać go w nowych rozmowach
- **Wizualizować pamięć**: Eksplorować relacje w grafie wiedzy Twojego agenta


## 🏗️ Co Zbudujesz

W tym poradniku stworzymy **Asystenta Kodowania** z trwałą pamięcią, który:

### 1. **Tworzenie Bazy Wiedzy**
   - Pobiera informacje o profilu i doświadczeniu programisty
   - Przetwarza zasady i najlepsze praktyki programowania w Pythonie
   - Przechowuje historyczne rozmowy między programistami a asystentami AI

### 2. **Rozmowy Świadome Sesji**
   - Utrzymuje kontekst w ramach wielu pytań w tej samej sesji
   - Automatycznie zapisuje każdą parę pytanie/odpowiedź dla efektywnego wyszukiwania
   - Dostarcza spójne, kontekstowe odpowiedzi na podstawie historii rozmowy

### 3. **Długoterminowa Pamięć**
   - Zachowuje ważne rozmowy w długoterminowej pamięci
   - Wydobywa istotne wspomnienia z bazy wiedzy i poprzednich sesji, aby wzbogacić nowe interakcje
   - Buduje rosnącą bazę wiedzy, która z czasem się ulepsza

### 4. **Inteligentne Wyszukiwanie Pamięci**
   - Wykorzystuje semantyczne wyszukiwanie oparte na grafach, aby znaleźć istotne informacje w całej przechowywanej wiedzy
   - Filtruje wyszukiwania według podgrup danych (informacje o programiście vs. zasady)
   - Łączy różne źródła danych, aby dostarczyć kompleksowe odpowiedzi


## 📋 Wymagania wstępne i konfiguracja

### Wymagania systemowe

Przed rozpoczęciem upewnij się, że masz:

1. **Środowisko Python**
   - Python 3.9 lub nowszy
   - Wirtualne środowisko (zalecane)
   
2. **Redis Cache** (Wymagane do zarządzania sesjami)
   - Lokalny Redis: `docker run -d -p 6379:6379 redis`
   - Lub skorzystaj z zarządzanej usługi Redis
   
3. **Dostęp do API LLM**
   - Klucz API OpenAI lub innych dostawców (zobacz [dokumentację](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Konfiguracja bazy danych**
   - Domyślnie nie jest wymagana żadna konfiguracja. Cognee korzysta z baz danych opartych na plikach (LanceDB i Kuzu)
   - Opcjonalnie możesz skonfigurować Azure AI Search jako magazyn wektorów (zobacz [dokumentację](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Konfiguracja środowiska

Utwórz plik `.env` w katalogu swojego projektu z następującymi zmiennymi:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Zrozumienie Architektury Pamięci Cognee

### Jak działa Cognee

Cognee oferuje zaawansowany system pamięci, który wykracza poza prostą pamięć klucz-wartość:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Kluczowe elementy:

1. **Graf wiedzy**: Przechowuje jednostki, relacje i połączenia semantyczne  
2. **Wektory osadzeń**: Umożliwiają wyszukiwanie semantyczne w całej przechowywanej informacji  
3. **Pamięć sesji**: Utrzymuje kontekst rozmowy w trakcie i pomiędzy sesjami  
4. **NodeSets**: Organizują dane w logiczne kategorie dla ukierunkowanego wyszukiwania  

### Rodzaje pamięci w tym samouczku:

- **Pamięć trwała**: Długoterminowe przechowywanie w grafie wiedzy  
- **Pamięć sesji**: Tymczasowy kontekst rozmowy w pamięci podręcznej Redis  
- **Pamięć semantyczna**: Wyszukiwanie podobieństw oparte na wektorach w całych danych  


## 📦 Zainstaluj wymagane pakiety

Zainstaluj Cognee z obsługą Redis do zarządzania sesjami:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inicjalizacja środowiska i ładowanie bibliotek

Upewnij się, że:
1. Redis jest uruchomiony (np. za pomocą Dockera: `docker run -d -p 6379:6379 redis`)
2. Zmienne środowiskowe są ustawione przed zaimportowaniem modułów pamięci podręcznej
3. Jeśli to konieczne, zrestartuj kernel i uruchom komórki w odpowiedniej kolejności

Poniższa komórka:
1. Załaduje zmienne środowiskowe z pliku `.env`
2. Skonfiguruje Cognee z ustawieniami Twojego LLM
3. Włączy pamięć podręczną do zarządzania sesją
4. Zweryfikuje, czy wszystkie komponenty są poprawnie połączone


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfiguracja katalogów przechowywania

Cognee używa dwóch oddzielnych katalogów do swoich operacji:
- **Data Root**: Przechowuje zaimportowane dokumenty i przetworzone dane
- **System Root**: Zawiera bazę danych grafu wiedzy oraz metadane systemowe

W tym samouczku utworzymy odizolowane katalogi w następujący sposób:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Zresetuj stan pamięci

Zanim zaczniemy budować nasz system pamięci, upewnijmy się, że zaczynamy od zera.

> 💡 **Wskazówka**: Możesz pominąć ten krok, jeśli chcesz zachować istniejące wspomnienia z poprzednich uruchomień tego notatnika.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Część 1: Tworzenie bazy wiedzy

### Źródła danych dla naszego asystenta programisty

Wykorzystamy trzy rodzaje danych, aby stworzyć kompleksową bazę wiedzy:

1. **Profil programisty**: Osobista wiedza i doświadczenie techniczne  
2. **Najlepsze praktyki w Pythonie**: Zasady Zen of Python z praktycznymi wskazówkami  
3. **Historyczne rozmowy**: Poprzednie sesje pytań i odpowiedzi między programistami a asystentami AI  

Ta różnorodność danych pozwala naszemu agentowi:  
- Zrozumieć techniczny kontekst użytkownika  
- Stosować najlepsze praktyki w rekomendacjach  
- Uczyć się na podstawie wcześniejszych udanych interakcji  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Przetwarzanie danych na graf wiedzy

Teraz przekształcimy nasz surowy tekst w uporządkowaną pamięć. Ten proces:

1. **Dodaje dane do NodeSets**: Organizuje informacje w logiczne kategorie
   - `developer_data`: Profil dewelopera i rozmowy
   - `principles_data`: Najlepsze praktyki i wytyczne dotyczące Pythona

2. **Uruchamia pipeline Cognify**: Wydobywa encje, relacje i tworzy osadzenia
   - Identyfikuje kluczowe pojęcia
   - Tworzy semantyczne powiązania między powiązanymi informacjami
   - Generuje osadzenia wektorowe

Może to zająć chwilę, ponieważ LLM przetwarza tekst i buduje strukturę grafu:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Wizualizacja Grafu Wiedzy

Przyjrzyjmy się strukturze naszego grafu wiedzy. Wizualizacja pokazuje:
- **Węzły**: Jednostki wyodrębnione z tekstu (pojęcia, technologie, osoby)
- **Krawędzie**: Relacje i powiązania między jednostkami
- **Klastry**: Powiązane pojęcia pogrupowane według podobieństwa semantycznego

Otwórz wygenerowany plik HTML w przeglądarce, aby interaktywnie eksplorować graf:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Wzbogacanie pamięci za pomocą Memify

Funkcja `memify()` analizuje graf wiedzy i generuje inteligentne reguły dotyczące danych. Ten proces:
- Identyfikuje wzorce i najlepsze praktyki
- Tworzy praktyczne wytyczne na podstawie treści
- Ustanawia powiązania między różnymi obszarami wiedzy

Te reguły pomagają agentowi podejmować bardziej świadome decyzje podczas odpowiadania na pytania. Uchwycenie drugiej wizualizacji pozwala porównać, jak graf staje się gęstszy po wzbogaceniu.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Część 2: Inteligentne Wyszukiwanie w Pamięci

### Demonstracja 1: Integracja Wiedzy z Różnych Dokumentów

Teraz, gdy nasz graf wiedzy jest zbudowany, przetestujmy, jak Cognee łączy informacje z wielu źródeł, aby odpowiedzieć na złożone pytania.

Pierwsze zapytanie pokazuje:
- **Zrozumienie semantyczne**: Znajdowanie odpowiednich pojęć, nawet jeśli nie są one wyraźnie wspomniane
- **Kros-referencje**: Łączenie profilu dewelopera z zasadami Pythona
- **Rozumowanie kontekstowe**: Stosowanie najlepszych praktyk do konkretnych implementacji

### Demonstracja 2: Filtrowane Wyszukiwanie z NodeSets

Drugie zapytanie pokazuje, jak skierować wyszukiwanie na konkretne podzbiory grafu wiedzy:
- Wykorzystuje parametr `node_name`, aby przeszukiwać tylko w obrębie `principles_data`
- Dostarcza skoncentrowanych odpowiedzi z określonej dziedziny wiedzy
- Przydatne, gdy potrzebujesz informacji specyficznych dla danej domeny


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Część 3: Konfiguracja zarządzania sesją

### Włączanie pamięci rozmowy

Zarządzanie sesją jest kluczowe dla utrzymania kontekstu w trakcie wielu interakcji. Tutaj wykonamy:

1. **Inicjalizację kontekstu użytkownika**: Utwórz lub pobierz profil użytkownika do śledzenia sesji
2. **Konfigurację silnika pamięci podręcznej**: Połącz się z Redis w celu przechowywania historii rozmów
3. **Włączanie zmiennych sesji**: Skonfiguruj zmienne kontekstowe, które będą trwałe w trakcie zapytań

> ⚠️ **Ważne**: Wymaga uruchomionego Redis i ustawienia `CACHING=true` w Twoim środowisku


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Funkcja pomocnicza: Wyświetl historię sesji

Ta funkcja narzędziowa pozwala nam sprawdzić historię rozmów przechowywaną w Redis. Jest przydatna do:
- Debugowania zarządzania sesjami
- Weryfikacji, czy rozmowy są buforowane
- Zrozumienia, jaki kontekst jest dostępny dla agenta


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesja 1: Laboratorium wsparcia asynchronicznego — Pierwsze pytanie

Rozpocznij sesję `async-support-lab`, pytając o wzorce asyncio przyjazne dla telemetrii dla ogromnego web scrapera. Graf już zna asyncio, aiohttp i praktyki monitorowania, więc odpowiedź powinna odzwierciedlać wcześniejsze rozmowy, dostosowując się do nowego zapytania.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Sprawdź pamięć sesji 1 po pierwszej wymianie

Uruchomienie `show_history(session_1)` bezpośrednio po początkowym pytaniu potwierdza, że Cognee zapisał zarówno pytanie, jak i odpowiedź w Redis. Powinieneś zobaczyć jeden wpis z wytycznymi dotyczącymi współbieżności.


In [ ]:
await show_history(session_1)

## Sesja 1: Kontynuacja modeli danych

Następnie pytamy: "Kiedy powinienem wybrać dataclasses zamiast Pydantic?" używając tego samego identyfikatora sesji. Cognee powinno połączyć zasady Pythona z wcześniejszymi rozmowami o FastAPI, aby dostarczyć szczegółowych porad—pokazując, że kontekst przenosi się w ramach nazwanej sesji.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Potwierdź, że historia sesji 1 zawiera oba etapy

Kolejne wywołanie `show_history(session_1)` powinno ujawnić dwa wpisy Q&A. To odpowiada krokowi "odtwarzania pamięci" w laboratorium Mem0 i potwierdza, że dodatkowe etapy są dodawane do tego samego transkryptu.


In [ ]:
await show_history(session_1)

## Sesja 2: Wątek Przeglądu Projektu — Nowa Sesja

Aby pokazać izolację między wątkami, uruchamiamy `design-review-session` i prosimy o wskazówki dotyczące logowania dla przeglądów incydentów. Mimo że podstawowa baza wiedzy jest taka sama, nowy identyfikator sesji utrzymuje transkrypcje oddzielnie.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Przegląd Sesji 2 Historia

`show_history(session_2)` powinno wyświetlać jedynie parę zapytanie/odpowiedź z przeglądu projektu. Porównaj to z Sesją 1, aby podkreślić, jak Cognee utrzymuje niezależne transkrypty, jednocześnie wykorzystując wspólny graf wiedzy.


In [ ]:
await show_history(session_2)

## Podsumowanie

Gratulacje! Właśnie dodałeś swojemu asystentowi kodowania warstwę długoterminowej pamięci zasilaną przez Cognee.

W tym samouczku przekształciłeś surowe treści deweloperskie (kod, dokumentację, czaty) w graf + pamięć wektorową, którą Twój agent może przeszukiwać, analizować i stale ulepszać.

Czego się nauczyłeś:

1. **Od surowego tekstu do pamięci AI**: Jak Cognee przetwarza nieustrukturyzowane dane i zamienia je w inteligentną, przeszukiwalną pamięć, wykorzystując połączoną architekturę wektorową i grafu wiedzy.

2. **Wzbogacanie grafu za pomocą memify**: Jak wyjść poza podstawowe tworzenie grafu i użyć memify do dodawania wywnioskowanych faktów i bogatszych relacji do istniejącego grafu.

3. **Różne strategie wyszukiwania**: Jak przeszukiwać pamięć za pomocą różnych typów wyszukiwania (Q&A z uwzględnieniem grafu, uzupełnianie w stylu RAG, wglądy, surowe fragmenty, wyszukiwanie kodu itp.) w zależności od potrzeb Twojego agenta.

4. **Wizualna eksploracja**: Jak sprawdzać i debugować to, co stworzył Cognee, za pomocą wizualizacji grafu i interfejsu Cognee, aby faktycznie zobaczyć, jak wiedza jest zorganizowana.

5. **Pamięć uwzględniająca sesje**: Jak łączyć kontekst sesji z trwałą pamięcią semantyczną, aby agenci mogli pamiętać między uruchomieniami, nie ujawniając informacji między użytkownikami.


## Kluczowe informacje
1. Pamięć jako graf wiedzy wspierany przez osadzenia

    - **Strukturalne zrozumienie**: Cognee łączy magazyn wektorów i magazyn grafów, dzięki czemu Twoje dane są zarówno wyszukiwalne pod względem znaczenia, jak i powiązane relacjami. Domyślnie Cognee korzysta z baz danych opartych na plikach (LanceDB dla magazynu wektorów, Kuzu dla bazy grafów).

    - **Świadomość relacji w wyszukiwaniu**: Odpowiedzi mogą być oparte nie tylko na „podobnym tekście”, ale także na tym, jak jednostki są ze sobą powiązane.

    - **Żywa pamięć**: Warstwa pamięci ewoluuje, rośnie i pozostaje dostępna jako jeden połączony graf.

2. Tryby wyszukiwania i rozumowania
    - **Hybrydowe wyszukiwanie**: wyszukiwanie łączy podobieństwo wektorów, strukturę grafu i rozumowanie LLM, od surowego wyszukiwania fragmentów po odpowiadanie na pytania uwzględniające graf.

    - **Dostosowanie trybu do zadania**: Używaj trybów w stylu uzupełniania, gdy potrzebujesz odpowiedzi w języku naturalnym, oraz trybów fragmentów/podsumowań/grafów, gdy Twój agent potrzebuje surowego kontekstu lub chce prowadzić własne rozumowanie.

3. Spersonalizowani agenci świadomi sesji
    - **Kontekst sesji + pamięć długoterminowa**: Cognee oddziela krótkoterminowy kontekst „wątku” od długoterminowej pamięci na poziomie użytkownika lub organizacji.

## Zastosowania w rzeczywistości

1. **Pionowe agenty AI**

    Wykorzystaj wzorzec z tego notatnika, aby stworzyć inteligentne asystenty domenowe, które opierają się na Cognee jako rdzeniu wyszukiwania i rozumowania:

- **Asystenci dla programistów**: Przegląd kodu, analiza incydentów i asystenci architektury, którzy przeszukują kod, API, dokumenty projektowe i zgłoszenia jako jeden graf pamięci.

- **Asystenci dla klientów**: Agenci wsparcia lub sukcesu, którzy korzystają z dokumentacji produktów, FAQ, notatek CRM i wcześniejszych zgłoszeń, oferując wyszukiwanie uwzględniające graf i odpowiedzi z cytatami.

- **Wewnętrzni eksperci**: Asystenci ds. polityki, prawa lub bezpieczeństwa, którzy rozumują na podstawie powiązanych zasad, wytycznych i decyzji historycznych, zamiast izolowanych plików PDF.

    Cognee jest wyraźnie pozycjonowane jako trwała, dokładna pamięć dla agentów AI, zapewniając żywy graf wiedzy, który zastępuje doraźne kombinacje magazynów wektorów i niestandardowego kodu grafów.

2. **Łączenie silosów danych w jedną pamięć**

    To samo podejście pomaga również stworzyć jednolitą warstwę pamięci z rozproszonych źródeł:

- **Od silosów do jednego grafu**: Importuj dane strukturalne (np. bazy danych) i niestrukturalne (np. dokumenty, czaty) do jednego grafu wspieranego przez osadzenia, zamiast oddzielnych indeksów dla każdego systemu.

- **Rozumowanie między źródłami z cytatami**: Przeprowadzaj wieloetapowe rozumowanie na podstawie wszystkiego—„łącz” logi, metryki i dokumenty za pomocą grafu—i nadal zwracaj odpowiedzi z uzasadnieniem i źródłem.

- **Centra wiedzy**: W takich dziedzinach jak bankowość czy edukacja, Cognee jest już używane do łączenia plików PDF, systemów wewnętrznych i danych aplikacji w jeden graf wiedzy z wektorami, dzięki czemu agenci mogą odpowiadać na pytania z precyzyjnym, cytowanym kontekstem.

## Kolejne kroki

Zaimplementowałeś podstawową pętlę pamięci. Oto naturalne rozszerzenia, które możesz wypróbować samodzielnie (zobacz [dokumentację Cognee](https://docs.cognee.ai/) dla szczegółów):

1. **Eksperymentuj ze świadomością czasową**: Włącz temporal cognify, aby wyodrębniać wydarzenia i znaczniki czasu z tekstu.

2. **Wprowadź rozumowanie oparte na ontologii**: Zdefiniuj ontologię OWL dla swojej dziedziny. Skorzystaj z wsparcia ontologii Cognee, aby wyodrębnione jednostki i relacje były osadzone w tym schemacie, poprawiając jakość grafu i odpowiedzi specyficzne dla domeny.

3. **Dodaj pętlę opinii**: Pozwól Cognee dostosowywać wagi krawędzi grafu na podstawie rzeczywistych opinii użytkowników, aby wyszukiwanie poprawiało się z czasem, zamiast pozostawać statyczne.

4. **Dostosuj do personalizacji i zachowań sesji**: Używaj identyfikatorów użytkowników, najemców i zestawów danych, aby każda osoba lub zespół miała własny widok na wspólny silnik pamięci.

5. **Rozszerz na bardziej złożonych agentów**: Podłącz Cognee do frameworków agentów, aby stworzyć systemy wieloagentowe, które wszystkie korzystają z tej samej warstwy pamięci. *Microsoft Agent Framework x Cognee plugin już wkrótce.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż staramy się zapewnić dokładność, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji krytycznych zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
